### Imports

In [1]:
import pandas as pd
import json    
from tqdm.auto import tqdm

/workspaces/E-commerce_chatbot/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install groq
import os
os.environ["GROQ_API_KEY"] = "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"
from groq import Groq


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


###  Loading  the Data

In [3]:
json_file = '../data/products_data.json'
with open(json_file, 'r') as file:
    documents_raw = json.load(file)

In [4]:
json_file = '../data/products_data.json'

In [5]:
# Initialize an empty list to store processed product details
product_details = []

# Iterate through each product in the loaded JSON data
for product in documents_raw:
    product_id = product['ID']  # Assuming 'ID' exists in your JSON data

    # Append the product details to the list, adding an 'ID' field
    product_details.append({
        'ID': product_id,
        'Product_Name': product['Product_Name'],
        'Price': product['Price'],
        'Product_Url': product['Product_Url'],
        'About_Product': product['About_Product'],
        'Product_Specification': product['Product_Specification'],
        'Technical_Details': product['Technical_Details'],
        'Shipping_Weight': product['Shipping_Weight'],
        'Product_Dimensions': product['Product_Dimensions'],
        'Image': product['Image'],
        'Variants': product['Variants'],
        'Color': product['Color'],
        'Product_Description': product['Product_Description'],
        'Category': product['Category']
    })

# Print the results
print(f"Loaded product details from '{json_file}' with {len(product_details)} entries.")


Loaded product details from '../data/products_data.json' with 9895 entries.


In [6]:
product_details[3]

{'ID': '18018b6bc416dab347b1b7db79994afa',
 'Product_Name': 'Guillow Airplane Design Studio with Travel Case Building Kit',
 'Price': '$28.91',
 'Product_Url': 'https://www.amazon.com/Guillow-Airplane-Design-Studio-Building/dp/B076Y2SNHM',
 'About_Product': "Make 8 different Planes at one time. | Experiment with different designs and learn about flight. | All contained in Part trays inside an attractive box with carry handle! | Guillow's - your one source for flying toys since 1926. | Made in the USA.",
 'Product_Specification': 'ProductDimensions:3.5x6.2x13inches|ItemWeight:13.4ounces|ShippingWeight:13.4ounces(Viewshippingratesandpolicies)|DomesticShipping:ItemcanbeshippedwithinU.S.|InternationalShipping:ThisitemcanbeshippedtoselectcountriesoutsideoftheU.S.LearnMore|ASIN:B076Y2SNHM|Itemmodelnumber:142|Manufacturerrecommendedage:8yearsandup',
 'Technical_Details': 'Go to your orders and start the return Select the ship method Ship it! | Go to your orders and start the return Select the

### Creating Ground Truth Dataset

In [7]:
prompt_template = """
You emulate a customer who's interested in buying a product from our online shop. 
Formulate 5 questions this customer might ask based on the product's information. 
The record should contain the answer to the questions, and the questions should be complete and not too short. 
If possible, use as few words as possible from the record. Focus on rephrasing the information naturally.

The product record:

ID: {ID}
Product Name: {Product_Name}
Price: {Price}
Product URL: {Product_Url}
About Product: {About_Product}
Product Specification: {Product_Specification}
Technical Details: {Technical_Details}
Shipping Weight: {Shipping_Weight}
Product Dimensions: {Product_Dimensions}
Image: {Image}
Variants: {Variants}
Color: {Color}
Product Description: {Product_Description}
Category: {Category}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [8]:
prompt = prompt_template.format(**product)

In [9]:
product =product_details[6]

In [10]:
product

{'ID': '40d3cd16b41970ae6872e914aecf2c8e',
 'Product_Name': "Rubie's Child's Pokemon Deluxe Pikachu Costume, X-Small",
 'Price': '$29.12',
 'Product_Url': 'https://www.amazon.com/Rubies-Deluxe-Pikachu-Pokemon-Costume/dp/B07BZJM592',
 'About_Product': "Make sure this fits by entering your model number. | Polyester | Deluxe Pokemon child's Pikachu costume features a fleece jumpsuit with attached tail and headpiece with attached ears and printed face | Officially licensed Pokemon costume, look for trademark on packaging and label to help assure you've received authentic safety-tested item | Important: Costumes are sized differently than apparel, use the Rubie's Child Size Chart, do not choose based on child's age or clothing size | Group costume fun: coordinate with additional Pokemon costumes and accessories available from Rubie's | Family-owned, family-focused, and based in the U.S.A. Since 1951, Rubie's has classic and licensed costumes and accessories in sizes and styles for your enti

In [11]:
client = Groq()
response = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )
    
json_response = response.choices[0].message.content
json_response

'["Can the Hasegawa Lucano Step Ladder be folded and stood on its own for easy storage?", \n"How much weight can the Hasegawa Lucano Step Ladder safely hold?", \n"What is the primary material used to make the Hasegawa Lucano Step Ladder?", \n"Are there any features on the steps of the Hasegawa Lucano Step Ladder to prevent slipping?", \n"What are the dimensions of the Hasegawa Lucano Step Ladder when it is open and in use?"]'

In [12]:
import hashlib
from tqdm.auto import tqdm
import pandas as pd
import os
from groq import Groq

# Function to generate questions using Groq
def generate_questions(product):
    prompt = prompt_template.format(**product)
    
    client = Groq()
    response = client.chat.completions.create(
        model="mixtral-8x7b-32768",   
        messages=[{"role": "user", "content": prompt}]
    )
    
    json_response = response.choices[0].message.content
    return json_response

In [ ]:
results = {}
for product in tqdm(product_details): 
    product_id = product['ID']
    if product_id in results:
        continue

    questions = generate_questions(product)
    results[product_id] = questions

  0%|▍                                                                                               | 49/9895 [09:29<32:35:14, 11.91s/it]